In [1]:
import numpy as np
from scipy.optimize import fsolve, minimize

In [2]:
n_years = 13  # Number of time steps (years)
delta_t = 1  # Time step size (years)
face_value = 100.0  # Face value of the zero-coupon bond
spot_rates = np.array([7.3, 7.62, 8.1, 8.45, 9.2, 9.64, 10.12, 10.45, 
                       10.75, 11.22, 11.55, 11.92, 12.2, 12.32])
a= np.array([7.29999752722836, 7.90126469922232, 8.97605012230675, 9.36502699052018, 12.0093837510071, 
             11.5731272238594, 12.6594489019931, 12.3466327689557, 12.6637447577049, 14.8572495468136, 
             14.1764745547454, 15.2147712367476, 14.7056308409242, 13.0195231437149])
b = np.array([0.01] * (n_years+1))  # Constant volatility of 1%
p=0.5 # Assume risk neutral probability is 0.5
q=1-p

In [3]:
n_years = 9  # Number of time steps (years)
spot_rates = np.array([3, 3.1, 3.2, 3.3, 3.4, 3.5, 3.55, 3.6, 
                       3.65, 3.7])
b = np.array([0.1] * (n_years+1))  # Constant volatility of 1%
p=0.5 # Assume risk neutral probability is 0.5
q=1-p

In [4]:
def build_short_rate_lattice(a, b, n_years):
    # build short rate lattice
    short_rate_lattice=[]
    for i in range(n_years+1):
        short_rate_lattice.append([0.0]*(i+1))
        
    for i in range(0, len(short_rate_lattice)):
        for j in range(0, len(short_rate_lattice[i])):
            short_rate_lattice[i][j]=a[i]*np.exp(b[i]*j)
    return short_rate_lattice

def build_elementary_prices(short_rate_lattice, p, q, n_years):
    # build elementary prices
    elementary_prices=[]
    for i in range(n_years+2):
        elementary_prices.append([0.0]*(i+1))

    for i in range(0, len(elementary_prices)):
        for j in range(0, len(elementary_prices[i])):
            if i==0:
                elementary_prices[i][j]=1
            else:
                if j==0:
                    elementary_prices[i][j]=q*elementary_prices[i-1][j]/(1+short_rate_lattice[i-1][j]/100)
                elif j==i:
                    elementary_prices[i][j]=p*elementary_prices[i-1][j-1]/(1+short_rate_lattice[i-1][j-1]/100)
                else:
                    elementary_prices[i][j]=p*elementary_prices[i-1][j-1]/(1+short_rate_lattice[i-1][j-1]/100) + \
                                            q*elementary_prices[i-1][j]/(1+short_rate_lattice[i-1][j]/100)
    return elementary_prices


def calibration_function(a, spot_rates, b, p, q, n_years):

    short_rate_lattice = build_short_rate_lattice(a, b, n_years)
    elementary_prices=build_elementary_prices(short_rate_lattice, p, q, n_years)
    
    BDT_ZCB_price = [sum(i) for i in elementary_prices]
    BDT_spot_rates = [((1 / i[1]) ** (1 / i[0])-1)*100 for i in zip(range(1, n_years + 2), BDT_ZCB_price[1:n_years + 2])]
    BDT_spot_rates = np.array(BDT_spot_rates)
    error=sum((BDT_spot_rates-spot_rates)**2)
    return error

In [5]:
computed_a=minimize(calibration_function, np.array([1]*(n_years+1)), 
                    args=(spot_rates, b, p, q, n_years))['x']

In [6]:
short_rate_lattice = build_short_rate_lattice(computed_a, b, n_years)

In [7]:
print("BDT Short Rate Lattice:")
for i, rates in enumerate(short_rate_lattice):
    print(f"Time {i :.1f}: {[round(r, 4) for r in rates]}")


BDT Short Rate Lattice:
Time 0.0: [np.float64(3.0)]
Time 1.0: [np.float64(3.0405), np.float64(3.3602)]
Time 2.0: [np.float64(3.0697), np.float64(3.3926), np.float64(3.7494)]
Time 3.0: [np.float64(3.0891), np.float64(3.414), np.float64(3.773), np.float64(4.1698)]
Time 4.0: [np.float64(3.0991), np.float64(3.425), np.float64(3.7852), np.float64(4.1833), np.float64(4.6232)]
Time 5.0: [np.float64(3.1012), np.float64(3.4274), np.float64(3.7878), np.float64(4.1862), np.float64(4.6265), np.float64(5.113)]
Time 6.0: [np.float64(2.8366), np.float64(3.1349), np.float64(3.4646), np.float64(3.829), np.float64(4.2317), np.float64(4.6767), np.float64(5.1686)]
Time 7.0: [np.float64(2.7669), np.float64(3.0579), np.float64(3.3795), np.float64(3.7349), np.float64(4.1277), np.float64(4.5619), np.float64(5.0416), np.float64(5.5719)]
Time 8.0: [np.float64(2.6974), np.float64(2.9811), np.float64(3.2947), np.float64(3.6412), np.float64(4.0241), np.float64(4.4473), np.float64(4.915), np.float64(5.432), np.floa

In [8]:
# value a payer swaption
k=3.9 # the fixed rate is 11.65%
n=10 # the swap matures at n=10
t=3 # the option expires at t=2

prices=[]
for i in range(n):
    prices.append([0.0]*(i+1))
prices[-1]=[(i-k)/100/(1+i/100) for i in short_rate_lattice[n-1]]

for i in range(len(prices)-1, -1, -1):
    if i==n-1: # for the last period
        prices[i]=[(i-k)/100/(1+i/100) for i in short_rate_lattice[n-1]] 
    elif i>t: # for the period option exerised and the swap is active
        for j in range(len(prices[i])):
            prices[i][j]=((p*prices[i+1][j] # from the upper node
                           +q*prices[i+1][j+1] # from the lower node
                           +(short_rate_lattice[i][j]-k)/100 # from the intrinsic value
                           )/(1+short_rate_lattice[i][j]/100))
    elif i==t: # for the period option is exercised
        for j in range(len(prices[i])):
            prices[i][j]=max(0,(( # option payoff is positive
                            p*prices[i+1][j] # from the upper node
                           +q*prices[i+1][j+1] # from the lower node
                           +(short_rate_lattice[i][j]-k)/100 # from the intrinsic value
                           )/(1+short_rate_lattice[i][j]/100)))
    else: # for the period option is not yet exercised
        for j in range(len(prices[i])):
            prices[i][j]=((p*prices[i+1][j] # from the upper node
                           +q*prices[i+1][j+1] # from the lower node
                           )/(1+short_rate_lattice[i][j]/100))
            
price=prices[0][0]
print(f"""The payer swaption valuation price is: {price*face_value:.4f}""")

The payer swaption valuation price is: 0.8097


In [9]:
# Valuation of defaultable zero coupon bond with recovery
n_years=9
r_00=0.05
u=1.1
d=0.9
p=0.5
q=1-p
a=0.01
b=1.01
face_value=100
R=0.2 # Recovery rate


In [10]:
def build_interest_rate_tree(n_years, r_00, u, d):
    rate_tree=[]
    for i in range(n_years+1):
            rate_tree.append([0.0]*(i+1))

    for i in range(n_years+1):
        for j in range(i+1):
            if i==0:
                rate_tree[i][j]=r_00
            else:
                rate_tree[i][j]=rate_tree[i-1][j]*d if j==0 else rate_tree[i-1][j-1]*u
    return rate_tree

rate_tree=build_interest_rate_tree(n_years, r_00, u, d)
print("Rate Tree:")
for i, rates in enumerate(rate_tree):
    print(f"Time {i :.1f}: {[round(r, 4) for r in rates]}")


Rate Tree:
Time 0.0: [0.05]
Time 1.0: [0.045, 0.055]
Time 2.0: [0.0405, 0.0495, 0.0605]
Time 3.0: [0.0365, 0.0446, 0.0545, 0.0666]
Time 4.0: [0.0328, 0.0401, 0.049, 0.0599, 0.0732]
Time 5.0: [0.0295, 0.0361, 0.0441, 0.0539, 0.0659, 0.0805]
Time 6.0: [0.0266, 0.0325, 0.0397, 0.0485, 0.0593, 0.0725, 0.0886]
Time 7.0: [0.0239, 0.0292, 0.0357, 0.0437, 0.0534, 0.0652, 0.0797, 0.0974]
Time 8.0: [0.0215, 0.0263, 0.0322, 0.0393, 0.048, 0.0587, 0.0717, 0.0877, 0.1072]
Time 9.0: [0.0194, 0.0237, 0.0289, 0.0354, 0.0432, 0.0528, 0.0646, 0.0789, 0.0965, 0.1179]


In [11]:
def build_hazrd_tree(n_years, a, b):
        hazard_tree=[]
        for i in range(n_years+1):
                hazard_tree.append([0.0]*(i+1))

        # hazrd rate this time is h_ij=a*b^{j-i/2}
        for i in range(n_years+1):
                for j in range(i+1):
                        hazard_tree[i][j]=a*b**(j-i/2)
        return hazard_tree

hazard_tree=build_hazrd_tree(n_years, a, b)
print("Hazard Tree:")
for i, rates in enumerate(hazard_tree):
    print(f"Time {i :.1f}: {[round(r, 4) for r in rates]}")

Hazard Tree:
Time 0.0: [0.01]
Time 1.0: [0.01, 0.01]
Time 2.0: [0.0099, 0.01, 0.0101]
Time 3.0: [0.0099, 0.01, 0.01, 0.0102]
Time 4.0: [0.0098, 0.0099, 0.01, 0.0101, 0.0102]
Time 5.0: [0.0098, 0.0099, 0.01, 0.01, 0.0102, 0.0103]
Time 6.0: [0.0097, 0.0098, 0.0099, 0.01, 0.0101, 0.0102, 0.0103]
Time 7.0: [0.0097, 0.0098, 0.0099, 0.01, 0.01, 0.0102, 0.0103, 0.0104]
Time 8.0: [0.0096, 0.0097, 0.0098, 0.0099, 0.01, 0.0101, 0.0102, 0.0103, 0.0104]
Time 9.0: [0.0096, 0.0097, 0.0098, 0.0099, 0.01, 0.01, 0.0102, 0.0103, 0.0104, 0.0105]


In [12]:
def build_defaultable_bond_tree(n_years, face_value, hazard_tree, rate_tree, R, p, q):
    zcb_prices=[]
    for i in range(n_years+1):
        zcb_prices.append([0.0]*(i+1))

    for i in range(n_years, -1, -1):
        for j in range(i+1):
            if i==n_years:
                zcb_prices[i][j]=(face_value*(
                    (1-hazard_tree[i][j]) # not default part
                    + R*(hazard_tree[i][j]) # default part
                    ))/(1+rate_tree[i][j])
            else:
                zcb_prices[i][j]=(
                    p*zcb_prices[i+1][j+1]*( # from upper node
                    (1-hazard_tree[i][j]) # not default part
                )+
                q*zcb_prices[i+1][j]*( # from lower node
                    (1-hazard_tree[i][j]) # not default part
                )+
                face_value*R*(hazard_tree[i][j]) # default part
                )/(1+rate_tree[i][j])
    return zcb_prices

zcb_prices=build_defaultable_bond_tree(n_years, face_value, hazard_tree, rate_tree, R, p, q)

print("ZCB Tree:")
for i, rates in enumerate(zcb_prices):
    print(f"Time {i :.1f}: {[round(r, 4) for r in rates]}")

print(f"""
      The defaultable zero coupon bond price is: {zcb_prices[0][0]:.4f}
""")

ZCB Tree:
Time 0.0: [57.2169]
Time 1.0: [63.0337, 57.9314]
Time 2.0: [68.5949, 64.0674, 59.0026]
Time 3.0: [73.8379, 69.9356, 65.4963, 60.5171]
Time 4.0: [78.72, 75.4636, 71.7053, 67.416, 62.5866]
Time 5.0: [83.2167, 80.6018, 77.5465, 74.0073, 69.9504, 65.3583]
Time 6.0: [87.3194, 85.3217, 82.963, 80.1959, 76.9751, 73.2618, 69.0295]
Time 7.0: [91.032, 89.6123, 87.9212, 85.9161, 83.5518, 80.7829, 77.567, 73.869]
Time 8.0: [94.3684, 93.4776, 92.4087, 91.1299, 89.6055, 87.7966, 85.662, 83.16, 80.2509]
Time 9.0: [97.3493, 96.9324, 96.4293, 95.8228, 95.0934, 94.2182, 93.1716, 91.9249, 90.447, 88.7052]

      The defaultable zero coupon bond price is: 57.2169

